## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
##
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
##
import time
import random
from datetime import datetime

#visulaization modules
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Historic Data

In [2]:
data_old = pd.read_csv('export_GDPR_fines_last.csv')

In [3]:
data_old.head()

,Unnamed: 0,ID,Country,Date,Amount,Controller_Processor,Quoted_Art,Type
0,0,ETid-1340,GERMANY,2021,Unknown,Private individual,Art. 5 (1) c) GDPR,Non-compliance with general data processing pr...
1,1,ETid-1339,GERMANY,2021,"Fine amount between EUR 100 and EUR 1,000",Private individual,Art. 6 GDPR,Insufficient legal basis for data processing
2,2,ETid-1338,GERMANY,2021,Unknown,Unknown,Art. 32 GDPR,Insufficient technical and organisational meas...
3,3,ETid-1337,GERMANY,2021,Unknown,Unknown,Art. 32 GDPR,Insufficient technical and organisational meas...
4,4,ETid-1336,GERMANY,2021,Unknown,Company,"Art. 25 GDPR, Art. 32 GDPR",Insufficient technical and organisational meas...


## Scraping data

In [5]:
driver = webdriver.Chrome()
driver.get("https://www.enforcementtracker.com/")
driver.maximize_window()

In [7]:
#getting last ID vs historic data
last_current_id = driver.find_element(By.XPATH, '//*[@id="penalties"]/tbody/tr[1]/td[2]/a').text
last_current_id = last_current_id.replace('ETid-', '')

last_historic_id = data_old.loc[0]['ID'].replace('ETid-', '')

delta = int(last_current_id) - int(last_historic_id)

pages_to_load = delta//100 + 1

In [8]:
select = Select(driver.find_element(By.XPATH, '//*[@id="penalties_length"]/label/select'))
# select by visible text
select.select_by_visible_text('100')

In [9]:
headers = []
for i in range(2,14):
    elem = driver.find_element(By.XPATH, '//*[@id="penalties"]/thead/tr[1]/th['+str(i)+']')
    if elem.text != "":
        headers.append(elem.text)

In [10]:
data = []
content = []

for page in range(1, pages_to_load + 1):

    print('Page ' + str(page) + ' / ' + str(pages_to_load) + ' being scrapped.')
    
    for i in range(1,101):
        
        for j in range(2,13):
            
            elem = driver.find_element(By.XPATH, '//*[@id="penalties"]/tbody/tr['+str(i)+']/td['+str(j)+']')
            
            if elem.text != '':
                content.append(elem.text)
    
        data.append(content)
        content = []
    
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
    ActionChains(driver).click(driver.find_element(By.XPATH, '//*[@id="penalties_paginate"]/span/a['+str(page+1)+']')).perform()        
    
    print("Page " + str(page) + ' / ' + str(pages_to_load) + " done !")

    time.sleep(random.randint(2, 7))

Page 1 / 3 en cours de chargement.
Page 1 / 3 récupérée !
Page 2 / 3 en cours de chargement.
Page 2 / 3 récupérée !
Page 3 / 3 en cours de chargement.
Page 3 / 3 récupérée !


## Loading Data

In [32]:
df_delta = pd.DataFrame(data, columns = ['ID','Country','Date','Amount','Controller_Processor','Quoted_Art','Type','Source'])

In [33]:
df_delta.shape

(300, 8)

In [34]:
display(df_delta.head(5))
display(df_delta.info())

,ID,Country,Date,Amount,Controller_Processor,Quoted_Art,Type,Source
0,ETid-1542,ITALY,2022-11-10,"20,000",Sportitalia,"Art. 5 (1) a) GDPR, Art. 9 GDPR, Art. 13 GDPR,...",Non-compliance with general data processing pr...,link link
1,ETid-1541,ITALY,2022-11-24,"1,000,000",Areti spa,"Art. 5 (1) d), e) GDPR, Art. 5 (2) GDPR, Art. ...",Non-compliance with general data processing pr...,link link
2,ETid-1540,ROMANIA,2022-12-27,"3,000",Kaufland Romania SCS,"Art. 29 GDPR, Art. 32 (1) b) GDPR, Art. 32 (2)...",Insufficient technical and organisational meas...,link
3,ETid-1539,ROMANIA,2022-12-22,"10,000",SUDREZIDENȚIAL Broker S.R.L.,Art. 32 (4) GDPR,Insufficient technical and organisational meas...,link
4,ETid-1538,ITALY,2022-11-10,"4,000",Villafranca di Verona municipality,"Art. 5 (1) a), c) GDPR, Art. 6 GDPR, Art. 2-te...",Non-compliance with general data processing pr...,link


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    300 non-null    object
 1   Country               300 non-null    object
 2   Date                  300 non-null    object
 3   Amount                300 non-null    object
 4   Controller_Processor  300 non-null    object
 5   Quoted_Art            300 non-null    object
 6   Type                  300 non-null    object
 7   Source                300 non-null    object
dtypes: object(8)
memory usage: 18.9+ KB


None

In [35]:
total_df = pd.concat([df_delta.drop('Source', axis = 1), data_old.drop(['Unnamed: 0'], axis = 1)], ignore_index=True)

total_df = total_df.drop_duplicates(subset = 'ID')

total_df.head()

,ID,Country,Date,Amount,Controller_Processor,Quoted_Art,Type
0,ETid-1542,ITALY,2022-11-10,"20,000",Sportitalia,"Art. 5 (1) a) GDPR, Art. 9 GDPR, Art. 13 GDPR,...",Non-compliance with general data processing pr...
1,ETid-1541,ITALY,2022-11-24,"1,000,000",Areti spa,"Art. 5 (1) d), e) GDPR, Art. 5 (2) GDPR, Art. ...",Non-compliance with general data processing pr...
2,ETid-1540,ROMANIA,2022-12-27,"3,000",Kaufland Romania SCS,"Art. 29 GDPR, Art. 32 (1) b) GDPR, Art. 32 (2)...",Insufficient technical and organisational meas...
3,ETid-1539,ROMANIA,2022-12-22,"10,000",SUDREZIDENȚIAL Broker S.R.L.,Art. 32 (4) GDPR,Insufficient technical and organisational meas...
4,ETid-1538,ITALY,2022-11-10,"4,000",Villafranca di Verona municipality,"Art. 5 (1) a), c) GDPR, Art. 6 GDPR, Art. 2-te...",Non-compliance with general data processing pr...


In [36]:
total_df.shape

(1542, 7)

In [37]:
#check IDs unicity
if total_df['ID'].value_counts().sort_values(ascending = False)[0] < 2:
    print('No duplicates !')
else:
    print('Dedup to be done !')

No duplicates !


In [38]:
#export for saving
total_df.to_csv('export_GDPR_fines_last.csv')

## Data Cleaning

In [51]:
fines_df = pd.read_csv('export_GDPR_fines_last.csv').drop('Unnamed: 0', axis = 1)

In [52]:
display(fines_df.head())
display(fines_df.info())

,ID,Country,Date,Amount,Controller_Processor,Quoted_Art,Type
0,ETid-1542,ITALY,2022-11-10,"20,000",Sportitalia,"Art. 5 (1) a) GDPR, Art. 9 GDPR, Art. 13 GDPR,...",Non-compliance with general data processing pr...
1,ETid-1541,ITALY,2022-11-24,"1,000,000",Areti spa,"Art. 5 (1) d), e) GDPR, Art. 5 (2) GDPR, Art. ...",Non-compliance with general data processing pr...
2,ETid-1540,ROMANIA,2022-12-27,"3,000",Kaufland Romania SCS,"Art. 29 GDPR, Art. 32 (1) b) GDPR, Art. 32 (2)...",Insufficient technical and organisational meas...
3,ETid-1539,ROMANIA,2022-12-22,"10,000",SUDREZIDENȚIAL Broker S.R.L.,Art. 32 (4) GDPR,Insufficient technical and organisational meas...
4,ETid-1538,ITALY,2022-11-10,"4,000",Villafranca di Verona municipality,"Art. 5 (1) a), c) GDPR, Art. 6 GDPR, Art. 2-te...",Non-compliance with general data processing pr...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1542 entries, 0 to 1541
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    1542 non-null   object
 1   Country               1542 non-null   object
 2   Date                  1542 non-null   object
 3   Amount                1542 non-null   object
 4   Controller_Processor  1542 non-null   object
 5   Quoted_Art            1542 non-null   object
 6   Type                  1542 non-null   object
dtypes: object(7)
memory usage: 84.5+ KB


None

In [53]:
def prepare_data(df):
    
    df['ID'] = df['ID'].str.replace('ETid-', '')
    df['ID'] = df['ID'].astype(int)

    df['Amount'] = df['Amount'].str.replace(' ','').str.replace(r'[a-zA-Z,-]','').replace(r'^\s*$', 0, regex=True)
    df['Amount'] = df['Amount'].fillna('0')
    df['Amount'] = df['Amount'].astype(int)
    
    df.loc[df['Amount'] == 0]['Amount'] = int(df['Amount'].median())
        
    df['Date'] = df['Date'].str.replace('Unknown','')
    df['Date'] = df['Date'].fillna(method = 'ffill')

    #df.loc[len(df['Date']) == 4]
        
    #treatment of article column
    df['Quoted_Art'] = df['Quoted_Art'].str.replace(r'[a-zA-Z.§é]','').str.replace(' ','').str.replace(r"\(.*\)",'')
    df = df.join(pd.get_dummies(df['Quoted_Art'].str.split(',').apply(pd.Series).stack(), prefix = 'Art').sum(level=0))    
    
    df['VIOLATED_ARTICLES'] = df[[x for x in df.columns if "Art" in x]].sum(axis = 1)
                        
    return df

In [54]:
data = prepare_data(fines_df)

C:\Users\Valentin\AppData\Local\Temp\ipykernel_2440\3469497675.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Amount'] = df['Amount'].str.replace(' ','').str.replace(r'[a-zA-Z,-]','').replace(r'^\s*$', 0, regex=True)
C:\Users\Valentin\AppData\Local\Temp\ipykernel_2440\3469497675.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['Amount'] == 0]['Amount'] = int(df['Amount'].median())
C:\Users\Valentin\AppData\Local\Temp\ipykernel_2440\3469497675.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Quoted_Art'] = df['Quoted_Art'].str.replace(r'[a-zA-Z.§é]','').str.replace(' ','').str.replace(r"\(.*\)",'')
C:\Users\Valentin\Ap

In [55]:
data

,ID,Country,Date,Amount,Controller_Processor,Quoted_Art,Type,Art_,Art_10,Art_113,...,Art_48,Art_5,Art_58,Art_6,Art_7,Art_8,Art_82,Art_88,Art_9,VIOLATED_ARTICLES
0,1542,ITALY,2022-11-10,20000,Sportitalia,5,Non-compliance with general data processing pr...,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,1541,ITALY,2022-11-24,1000000,Areti spa,"5,12,15,24",Non-compliance with general data processing pr...,0,0,0,...,0,1,0,0,0,0,0,0,0,4
2,1540,ROMANIA,2022-12-27,3000,Kaufland Romania SCS,"29,32",Insufficient technical and organisational meas...,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,1539,ROMANIA,2022-12-22,10000,SUDREZIDENȚIAL Broker S.R.L.,32,Insufficient technical and organisational meas...,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1538,ITALY,2022-11-10,4000,Villafranca di Verona municipality,"5,6,2-",Non-compliance with general data processing pr...,0,0,0,...,0,1,0,1,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1537,5,BELGIUM,2019-05-28,2000,Mayor,"5,6",Insufficient legal basis for data processing,0,0,0,...,0,1,0,1,0,0,0,0,0,2
1538,4,AUSTRIA,2018-12-20,2200,Private person,"5,13",Insufficient legal basis for data processing,0,0,0,...,0,1,0,0,0,0,0,0,0,2
1539,3,AUSTRIA,2018-09-27,300,Private car owner,"5,6",Insufficient legal basis for data processing,0,0,0,...,0,1,0,1,0,0,0,0,0,2
1540,2,AUSTRIA,2018,1800,Kebab restaurant,"5,13,14",Insufficient legal basis for data processing,0,0,0,...,0,1,0,0,0,0,0,0,0,3


In [56]:
data.to_csv('export_GDPR_fines_cleaned.csv')